In [1]:
import base64
import dash
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from dash import html as html
from dash import dcc
import cv2                
import matplotlib.pyplot as plt                        
import os
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
import tensorflow as tf
from extract_bottleneck_features import *

Using TensorFlow backend.


In [2]:
# extract pre-trained face detector
face_cascade = cv2.CascadeClassifier('./haarcascades/haarcascade_frontalface_alt.xml')

In [3]:
dog_names=['Affenpinscher', 'Afghan hound', 'Airedale terrier', 'Akita', 'Alaskan malamute', 'American eskimo dog', 'American foxhound', 'American staffordshire terrier', 'American water spaniel', 'Anatolian shepherd dog', 'Australian cattle dog', 'Australian shepherd', 'Australian terrier', 'Basenji', 'Basset hound', 'Beagle', 'Bearded collie', 'Beauceron', 'Bedlington terrier', 'Belgian malinois', 'Belgian sheepdog', 'Belgian tervuren', 'Bernese mountain dog', 'Bichon frise', 'Black and tan coonhound', 'Black russian terrier', 'Bloodhound', 'Bluetick coonhound', 'Border collie', 'Border terrier', 'Borzoi', 'Boston terrier', 'Bouvier des flandres', 'Boxer', 'Boykin spaniel', 'Briard', 'Brittany', 'Brussels griffon', 'Bull terrier', 'Bulldog', 'Bullmastiff', 'Cairn terrier', 'Canaan dog', 'Cane corso', 'Cardigan welsh corgi', 'Cavalier king charles spaniel', 'Chesapeake bay retriever', 'Chihuahua', 'Chinese crested', 'Chinese shar-pei', 'Chow chow', 'Clumber spaniel', 'Cocker spaniel', 'Collie', 'Curly-coated retriever', 'Dachshund', 'Dalmatian', 'Dandie dinmont terrier', 'Doberman pinscher', 'Dogue de bordeaux', 'English cocker spaniel', 'English setter', 'English springer spaniel', 'English toy spaniel', 'Entlebucher mountain dog', 'Field spaniel', 'Finnish spitz', 'Flat-coated retriever', 'French bulldog', 'German pinscher', 'German shepherd dog', 'German shorthaired pointer', 'German wirehaired pointer', 'Giant schnauzer', 'Glen of imaal terrier', 'Golden retriever', 'Gordon setter', 'Great dane', 'Great pyrenees', 'Greater swiss mountain dog', 'Greyhound', 'Havanese', 'Ibizan hound', 'Icelandic sheepdog', 'Irish red and white setter', 'Irish setter', 'Irish terrier', 'Irish water spaniel', 'Irish wolfhound', 'Italian greyhound', 'Japanese chin', 'Keeshond', 'Kerry blue terrier', 'Komondor', 'Kuvasz', 'Labrador retriever', 'Lakeland terrier', 'Leonberger', 'Lhasa apso', 'Lowchen', 'Maltese', 'Manchester terrier', 'Mastiff', 'Miniature schnauzer', 'Neapolitan mastiff', 'Newfoundland', 'Norfolk terrier', 'Norwegian buhund', 'Norwegian elkhound', 'Norwegian lundehund', 'Norwich terrier', 'Nova scotia duck tolling retriever', 'Old english sheepdog', 'Otterhound', 'Papillon', 'Parson russell terrier', 'Pekingese', 'Pembroke welsh corgi', 'Petit basset griffon vendeen', 'Pharaoh hound', 'Plott', 'Pointer', 'Pomeranian', 'Poodle', 'Portuguese water dog', 'Saint bernard', 'Silky terrier', 'Smooth fox terrier', 'Tibetan mastiff', 'Welsh springer spaniel', 'Wirehaired pointing griffon', 'Xoloitzcuintli', 'Yorkshire terrier']

In [4]:
# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

In [5]:
from keras.applications.resnet50 import ResNet50

# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

W0305 11:25:18.857692  3752 module_wrapper.py:139] From C:\Users\toptech\.julia\conda\3\lib\site-packages\keras\backend\tensorflow_backend.py:68: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0305 11:25:18.857692  3752 module_wrapper.py:139] From C:\Users\toptech\.julia\conda\3\lib\site-packages\keras\backend\tensorflow_backend.py:508: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0305 11:25:18.877860  3752 module_wrapper.py:139] From C:\Users\toptech\.julia\conda\3\lib\site-packages\keras\backend\tensorflow_backend.py:3837: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0305 11:25:18.939687  3752 module_wrapper.py:139] From C:\Users\toptech\.julia\conda\3\lib\site-packages\keras\backend\tensorflow_backend.py:168: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0305 11:25:18.941683  3752 module_wrapper.py:13

In [6]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [7]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    global graph
    with graph.as_default():
        y=np.argmax(ResNet50_model.predict(img))
    return y

In [8]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151)) 

In [9]:
### TODO: Obtain bottleneck features from another pre-trained CNN.

bottleneck_features = np.load('./bottleneck_features/DogResnet50Data.npz')
train_Resnet50 = bottleneck_features['train']
valid_Resnet50 = bottleneck_features['valid']
test_Resnet50 = bottleneck_features['test']

In [10]:
### TODO: Define your architecture.

Resnet50_model = Sequential()
Resnet50_model.add(GlobalAveragePooling2D(input_shape=train_Resnet50.shape[1:]))
Resnet50_model.add(Dense(1024, activation='relu'))
Resnet50_model.add(Dropout(0.2))
Resnet50_model.add(Dense(133, activation='softmax'))

Resnet50_model.summary()

W0305 11:25:32.298829  3752 deprecation.py:506] From C:\Users\toptech\.julia\conda\3\lib\site-packages\keras\backend\tensorflow_backend.py:3144: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 133)               136325    
Total params: 2,234,501
Trainable params: 2,234,501
Non-trainable params: 0
_________________________________________________________________


In [11]:
### TODO: Compile the model.

Resnet50_model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

W0305 11:25:32.399023  3752 module_wrapper.py:139] From C:\Users\toptech\.julia\conda\3\lib\site-packages\keras\optimizers.py:757: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0305 11:25:32.433048  3752 module_wrapper.py:139] From C:\Users\toptech\.julia\conda\3\lib\site-packages\keras\backend\tensorflow_backend.py:3014: The name tf.log is deprecated. Please use tf.math.log instead.



In [12]:
### TODO: Load the model weights with the best validation loss.

Resnet50_model.load_weights('./saved_models/weights.best.Resnet50.hdf5')
graph = tf.get_default_graph()

In [13]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.

def Resnet50_predict_breed(img_path):
    # extract bottleneck features
    global graph
    with graph.as_default():
    # obtain predicted vector
        bottleneck_feature = extract_Resnet50(path_to_tensor(img_path))
        predicted_vector = Resnet50_model.predict(bottleneck_feature)
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]

In [14]:
### TODO: Write your algorithm.
### Feel free to use as many code cells as needed.

def Dog_Identification(img_path):
    #Input: Path for an image to be classified
    #Output: Classification result which is either:
    #        - Dog breed for a detected dog
    #        - Resembling dog breed for a detected human 
    #        - Error in case neither dog nor human detected

    if dog_detector(img_path):
        dog_breed=Resnet50_predict_breed(img_path)
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return 'This seems to be a dog, Its predicted breed is ... '+ dog_breed
    elif face_detector(img_path):
        dog_breed=Resnet50_predict_breed(img_path)
        dog_breed=dog_breed.split('.')[-1]
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return 'This seems to be a human, The resembling dog breed is ... '+ dog_breed
    else :
        return 'Error! This does not look like a human or dog. Please try another image.'

## Dash Web App

In [15]:
FA = "https://use.fontawesome.com/releases/v5.8.1/css/all.css"

## Set the path of the 'assets' directory
assets='./Dash/assets'

external_stylesheets = [assets+'/'+'bWLwgP.css']
external_stylesheets = [dbc.themes.BOOTSTRAP, FA]


app = dash.Dash(__name__,
                title='Dog Breed Identification App',
                assets_folder=assets,assets_url_path=assets,
                external_stylesheets=external_stylesheets,
                meta_tags = [{"name": "viewport", "content": "width=device-width, height=device-height, initial-scale=1.0"}],
                suppress_callback_exceptions=True)


CONTENT_STYLE = {
    "margin-left": "10rem",
    "margin-right": "10rem",
    "padding": "2rem 1rem",
}

navbar = dbc.Navbar(
    [
        html.A(

            dbc.Row(
                [
        
                    dbc.Col(dbc.NavbarBrand("Dog Breed Identification App", className="ml-2",style={'color': '#FFFFFF','font-size': '30px',"margin-left": "0px"})),
                ],
                align="left",

            ),
            href="/"
        ),
        html.A(

            dbc.Row(
                [
                    dbc.Col(html.H3(id='Page-Nav-Title',children=' ', className='ml-4',style={'color': '#00d4ff','font-size': '20px', 'margin-left':'50px', 'width':'200px'})),
                ],
                align="center",

            ),

        ),
            ],
    color="#353535",
    dark=True,
)


content = html.Div(id="page-content", style=CONTENT_STYLE)

## Home Page

In [16]:
############################# Page01 ####################################
UPLOAD_DIRECTORY = "./temp"

if not os.path.exists(UPLOAD_DIRECTORY):
    os.makedirs(UPLOAD_DIRECTORY)

page0 = html.Div(style={'backgroundColor':'#f4f4f2'},
    children=[
    html.Div([
        html.Div(
            id="header",
            children=[
            html.H5(children='Please select the image to be processed',style={'margin-left':'10px', 'margin-top':'10px'}),
        ],style={'display': 'inline-block'}
        ),

    ]),

         html.Div([
            dcc.Upload(
                id='upload-image',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Image')
                ]),
                style={'align': 'center',
                    'width': '90%',
                    'height': '50px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px', 'margin-top':'20px','font-size': '15px', "font-weight": "bold", 'margin-left':'60px'
                },
                # Allow multiple files to be uploaded
                multiple=True
        )]),html.Div([

            dbc.Button(
            "Detect", id="process-button", className="d-grid col-2 mx-auto", color="secondary",
            style={'text-align': 'center','margin-top':'20px','margin-bottom':'20px','font-size': '20px', "font-weight": "bold", 'inline-block': 'block',"allign": "left"})]
             
         ),
        
        html.Div(children=html.Div(id='original-image',style={"width": "80%", "display": "inline-block", "padding": "0 0"})),
        html.H5(id='result',style={'margin-left':'10px', 'margin-top':'20px'}),

    ])


def save_file(name, content):
    """Decode and store a file uploaded with Plotly Dash."""
    data = content.encode("utf8").split(b";base64,")[1]
    with open(os.path.join(UPLOAD_DIRECTORY, name), "wb") as fp:
        fp.write(base64.decodebytes(data))
        return UPLOAD_DIRECTORY+'/'+name
     
@app.callback(
    [Output("original-image", "children"),Output("result", "children"), Output("process-button", "n_clicks")],
    [Input("upload-image", "filename"), Input("upload-image", "contents"), Input("process-button", "n_clicks")],
)

def update_output(uploaded_filename, uploaded_file_content, n_clicks):
    """Save uploaded files and regenerate the file list."""
    
    if ((n_clicks is None) | (n_clicks==0)):
        raise dash.exceptions.PreventUpdate

    elif uploaded_filename is not None and uploaded_file_content is not None:
        
        for name, data in zip(uploaded_filename, uploaded_file_content):
            image_path=save_file(name, data)
            classification_result=Dog_Identification(image_path)
        return [html.Div(children=html.Img(src=data, style={'allign': 'left', 'height':"auto",'width':"auto"})),classification_result, None]


## App Layout

In [ ]:
app.layout = html.Div([dcc.Location(id='url'), navbar,content, dcc.Store(id='original-image')])

@app.callback([Output('page-content', 'children'), Output('Page-Nav-Title','children')], [Input('url', 'pathname')])
def render_page_content(pathname):
    return page0,'' 

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/



I0305 11:25:34.931489  3752 dash.py:2021] Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


I0305 11:25:34.973328  3752 _internal.py:225]  * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
I0305 11:25:59.785907  3972 _internal.py:225] 127.0.0.1 - - [05/Mar/2023 11:25:59] "GET / HTTP/1.1" 200 -
I0305 11:25:59.851294 10140 _internal.py:225] 127.0.0.1 - - [05/Mar/2023 11:25:59] "GET /Dash/assets/css/all.css?m=1614995166.0 HTTP/1.1" 200 -
I0305 11:25:59.880290  8520 _internal.py:225] 127.0.0.1 - - [05/Mar/2023 11:25:59] "GET /Dash/assets/css/bootstrap.min.css?m=1615738896.0 HTTP/1.1" 200 -
I0305 11:25:59.912278  5688 _internal.py:225] 127.0.0.1 - - [05/Mar/2023 11:25:59] "GET /Dash/assets/css/bWLwgP.css?m=1614855200.0 HTTP/1.1" 200 -
I0305 11:26:00.430871  3184 _internal.py:225] 127.0.0.1 - - [05/Mar/2023 11:26:00] "GET /_dash-layout HTTP/1.1" 200 -
I0305 11:26:00.434871  5200 _internal.py:225] 127.0.0.1 - - [05/Mar/2023 11:26:00] "GET /_dash-dependencies HTTP/1.1" 200 -
I0305 11:26:00.445753  3056 _internal.py:225] 127.0.0.1 - - [05/Mar/2023 11:26:00] "GET /_favicon.ico